In [3]:
import json
import os
import chromadb
import google.generativeai as genai
from statistics import median
from typing import List, Dict
from dotenv import load_dotenv
# ==========================================
# 0. CONFIGURATION
# ==========================================
# Export this in your terminal: export GOOGLE_API_KEY="your_key"
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY") 
DB_PATH = "./spark_rag_db" # Where vectors will be saved on disk

if not API_KEY:
    print("⚠️  WARNING: GOOGLE_API_KEY environment variable not set.")
else:
    genai.configure(api_key=API_KEY)

# ==========================================
# 1. PARSING LOGIC (Your Code)
# ==========================================

class SparkLogAnalysis:
    """A container object for all parsed summaries from a Spark event log."""
    def __init__(self, log_file_path):
        self.application_id = "unknown_app"
        self.log_file = os.path.basename(log_file_path)
        self.job_summaries = []
        self.stage_summaries = []
        self.application_summaries = {}

    def set_application_id(self, app_id):
        if app_id:
            self.application_id = app_id

    def add_summary(self, summary_dict):
        if not summary_dict: return
        event_type = summary_dict.get("event_type")
        summary_dict["application_id"] = self.application_id

        if event_type == "JobPerformanceSummary":
            self.job_summaries.append(summary_dict)
        elif event_type == "StagePerformanceSummary":
            self.stage_summaries.append(summary_dict)
        else:
            self.application_summaries[event_type] = summary_dict

    def to_dict(self):
        return {
            "application_id": self.application_id,
            "log_file": self.log_file,
            "job_summaries": self.job_summaries,
            "stage_summaries": self.stage_summaries,
            "application_summaries": self.application_summaries
        }

def analyze_spark_log(log_file_path):
    """Parses a single Spark event log."""
    analysis_object = SparkLogAnalysis(log_file_path)
    stage_data = {}
    job_data = {}
    executor_lifecycle = {}
    disk_related_executor_failures = {}
    blacklisted_executors = []

    try:
        with open(log_file_path, 'r') as f:
            for line in f:
                try:
                    event = json.loads(line)
                except json.JSONDecodeError:
                    continue

                event_type = event.get("Event")
                
                if event_type == "SparkListenerApplicationStart":
                    analysis_object.set_application_id(event.get("App ID"))
                
                elif event_type == "SparkListenerExecutorAdded":
                    executor_id, timestamp = event.get("Executor ID"), event.get("Timestamp")
                    executor_lifecycle.setdefault(executor_id, {})['add_time'] = timestamp

                elif event_type == "SparkListenerExecutorRemoved":
                    executor_id, timestamp = event.get("Executor ID"), event.get("Timestamp")
                    reason = event.get("Removed Reason", "")
                    executor_lifecycle.setdefault(executor_id, {})['remove_time'] = timestamp
                    if "no space left on device" in reason.lower() or "disk" in reason.lower():
                        disk_related_executor_failures.setdefault(reason, {"count": 0, "samples": []})
                        disk_related_executor_failures[reason]["count"] += 1
                        if len(disk_related_executor_failures[reason]["samples"]) < 3:
                            disk_related_executor_failures[reason]["samples"].append({"executor_id": executor_id, "timestamp": timestamp})

                elif event_type == "SparkListenerExecutorBlacklisted":
                    blacklisted_executors.append({"executor_id": event.get("Executor ID"), "timestamp": event.get("Timestamp"), "reason": f"Blacklisted for {event.get('Task ID')} task failures"})

                elif event_type == "SparkListenerJobStart":
                    job_id = event.get("Job ID")
                    job_data[job_id] = {"start_time": event.get("Submission Time")}

                elif event_type == "SparkListenerJobEnd":
                    job_id = event.get("Job ID")
                    if job_id in job_data:
                        duration_ms = event.get("Completion Time") - job_data[job_id]["start_time"]
                        job_result = event.get("Job Result", {}).get("Result")
                        analysis_object.add_summary({ "event_type": "JobPerformanceSummary", "job_id": job_id, "status": job_result, "duration_s": duration_ms / 1000, "summary": f"Job {job_id} finished with status {job_result} in {duration_ms / 1000:.2f} seconds." })
                        del job_data[job_id]

                elif event_type == "SparkListenerTaskEnd":
                    stage_id = event.get("Stage ID")
                    if stage_id is not None:
                        stage_data.setdefault(stage_id, {"tasks": [], "failed_task_count": 0})
                        if event.get("Task End Reason", {}).get("Reason") != "Success": stage_data[stage_id]["failed_task_count"] += 1
                        task_info, task_metrics = event.get("Task Info", {}), event.get("Task Metrics", {})
                        duration_ms = task_info.get("Finish Time", 0) - task_info.get("Launch Time", 0)
                        stage_data[stage_id]["tasks"].append({ "duration_ms": duration_ms, "jvm_gc_time_ms": task_metrics.get("JVM GC Time", 0), "executor_run_time_ms": task_metrics.get("Executor Run Time", 1), "executor_deserialize_time_ms": task_metrics.get("Executor Deserialize Time", 0), "shuffle_fetch_wait_time_ms": task_metrics.get("Shuffle Read Metrics", {}).get("Fetch Wait Time", 0), "memory_spilled_bytes": task_metrics.get("Memory Bytes Spilled", 0), "disk_spilled_bytes": task_metrics.get("Disk Bytes Spilled", 0), })

                elif event_type == "SparkListenerStageCompleted":
                    stage_id = event.get("Stage Info", {}).get("Stage ID")
                    if stage_id in stage_data and len(stage_data[stage_id]["tasks"]) > 0:
                        tasks, failed_task_count = stage_data[stage_id]["tasks"], stage_data[stage_id]["failed_task_count"]
                        task_durations = [t["duration_ms"] for t in tasks]
                        total_duration, total_gc_time, total_runtime = sum(task_durations), sum(t["jvm_gc_time_ms"] for t in tasks), sum(t["executor_run_time_ms"] for t in tasks)
                        total_deserialize_time, total_fetch_wait_time = sum(t["executor_deserialize_time_ms"] for t in tasks), sum(t["shuffle_fetch_wait_time_ms"] for t in tasks)
                        total_spilled_bytes = sum(t["memory_spilled_bytes"] for t in tasks) + sum(t["disk_spilled_bytes"] for t in tasks)
                        summary_parts, potential_issues = [f"Stage {stage_id} completed with {len(tasks)} tasks."], []
                        if failed_task_count > 0: summary_parts.append(f"It experienced {failed_task_count} task failures that required retries."); potential_issues.append("TASK_FAILURES")
                        max_duration, median_duration = max(task_durations), median(task_durations)
                        if max_duration > 3 * median_duration and max_duration > 20000: potential_issues.append("DATA_SKEW"); summary_parts.append(f"Detected potential data skew. Max task duration: {max_duration/1000:.2f}s, median: {median_duration/1000:.2f}s.")
                        if total_runtime > 0 and (total_gc_time / total_runtime) > 0.10: potential_issues.append("HIGH_GC_PRESSURE"); summary_parts.append(f"High JVM GC pressure detected ({ (total_gc_time / total_runtime) * 100:.1f}% of runtime).")
                        if total_spilled_bytes > 0: potential_issues.append("DATA_SPILL"); summary_parts.append(f"Detected data spilling to disk ({total_spilled_bytes / (1024*1024):.2f} MB).")
                        if total_duration > 0 and (total_fetch_wait_time / total_duration) > 0.25: potential_issues.append("SHUFFLE_BOTTLENECK"); summary_parts.append(f"Significant shuffle bottleneck, tasks spent {(total_fetch_wait_time / total_duration) * 100:.1f}% of time waiting for data.")
                        if total_runtime > 0 and (total_deserialize_time / total_runtime) > 0.15: potential_issues.append("DESERIALIZATION_BOTTLENECK"); summary_parts.append(f"High deserialization time ({(total_deserialize_time / total_runtime) * 100:.1f}% of runtime).")
                        analysis_object.add_summary({ "event_type": "StagePerformanceSummary", "stage_id": stage_id, "metrics": { "task_count": len(tasks), "failed_task_count": failed_task_count, "max_task_duration_ms": max_duration, "median_task_duration_ms": median_duration, "total_spilled_mb": total_spilled_bytes / (1024*1024) }, "summary": " ".join(summary_parts), "potential_issues": potential_issues if potential_issues else ["NONE"] })

        if disk_related_executor_failures:
            total_failures = sum(v['count'] for v in disk_related_executor_failures.values())
            analysis_object.add_summary({ "event_type": "ExecutorDiskFailureSummary", "total_failures": total_failures, "summary": f"A critical error occurred where {total_failures} executors failed to launch due to running out of disk space on the nodes.", "failures_by_reason": disk_related_executor_failures })
        
        short_lived_executors = []
        churn_threshold_ms = 60000 
        for exec_id, times in executor_lifecycle.items():
            if 'add_time' in times and 'remove_time' in times and (times['remove_time'] - times['add_time']) < churn_threshold_ms:
                short_lived_executors.append({"executor_id": exec_id, "lifespan_s": (times['remove_time'] - times['add_time']) / 1000})
        if short_lived_executors:
            avg_lifespan_s = sum(e['lifespan_s'] for e in short_lived_executors) / len(short_lived_executors)
            analysis_object.add_summary({ "event_type": "ExecutorChurnSummary", "total_churn_events": len(short_lived_executors), "summary": f"Detected {len(short_lived_executors)} executors with an average lifespan of just {avg_lifespan_s:.2f}s, indicating severe instability.", "average_lifespan_s": round(avg_lifespan_s, 2), "sample_churned_executors": short_lived_executors[:3] })

    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    return analysis_object

# ==========================================
# 2. VECTORIZATION (ChromaDB Integration)
# ==========================================

def get_chroma_collection():
    """Returns a persistent ChromaDB collection."""
    # Using PersistentClient to save data to disk so we don't re-index every run
    client = chromadb.PersistentClient(path=DB_PATH)
    # Using the default embedding model (all-MiniLM-L6-v2)
    return client.get_or_create_collection(name="spark_logs")

def ingest_logs(log_directory):
    """Parses logs and ingests them into ChromaDB."""
    collection = get_chroma_collection()
    
    # Check if we already have data (optional optimization)
    if collection.count() > 0:
        print(f"📚 Vector DB already contains {collection.count()} documents. Skipping ingestion.")
        print("To force re-ingestion, delete the 'spark_rag_db' folder.")
        return

    print(f"📂 Processing logs in: {log_directory}")
    all_analyses = []

    for filename in os.listdir(log_directory):
        full_path = os.path.join(log_directory, filename)
        if os.path.isfile(full_path):
            print(f"   - Parsing {filename}...")
            analysis = analyze_spark_log(full_path)
            if analysis:
                all_analyses.append(analysis)

    print("🧩 Vectorizing and storing data...")
    documents, metadatas, ids = [], [], []

    for analysis in all_analyses:
        # Flatten the object
        summaries = (analysis.job_summaries + 
                     analysis.stage_summaries + 
                     list(analysis.application_summaries.values()))

        for i, summary in enumerate(summaries):
            # 1. Text to Embed
            documents.append(summary["summary"])
            
            # 2. Metadata Cleaning (Chroma requires simple types)
            meta = summary.copy()
            del meta["summary"]
            
            # Convert list/dict fields to strings for metadata storage
            for key, val in meta.items():
                if isinstance(val, (list, dict)):
                    meta[key] = str(val)
            
            metadatas.append(meta)
            ids.append(f"{analysis.application_id}_{i}")

    if documents:
        collection.add(documents=documents, metadatas=metadatas, ids=ids)
        print(f"✅ Successfully ingested {len(documents)} log events into ChromaDB.")
    else:
        print("⚠️ No valid log events found to ingest.")

# ==========================================
# 3. LLM ORCHESTRATOR (The RAG Logic) - UPDATED
# ==========================================

def get_gemini_response(user_query, collection, app_id=None):
    """
    1. Retrieval: Search ChromaDB. If app_id is provided, filter by it.
       Otherwise, search across ALL applications.
    2. Augmentation: Build a prompt with the search results.
    3. Generation: Call Gemini API.
    """
    
    # 1. RETRIEVAL
    query_args = {
        "query_texts": [user_query],
        "n_results": 5
    }

    # Only apply the filter if an app_id is specifically requested
    if app_id:
        print(f"🔍 Searching logs specifically for App: {app_id}...")
        query_args["where"] = {"application_id": app_id}
    else:
        print(f"🔍 Searching across ALL application logs...")

    results = collection.query(**query_args)

    retrieved_docs = results['documents'][0]
    retrieved_meta = results['metadatas'][0]

    if not retrieved_docs:
        return "I couldn't find any log events matching your question."

    # 2. AUGMENTATION (Context Construction)
    context_parts = []
    for doc, meta in zip(retrieved_docs, retrieved_meta):
        # We add the App ID to the context so the LLM knows which app the log belongs to
        # when searching globally.
        current_app = meta.get('application_id', 'unknown')
        info_tag = f"[App: {current_app} | {meta.get('event_type', 'Event')}]"
        issue_tag = f"[Issues: {meta.get('potential_issues', 'None')}]"
        context_parts.append(f"{info_tag} {issue_tag}\nLog Summary: {doc}")

    context_str = "\n---\n".join(context_parts)

    # Dynamic System Prompt
    if app_id:
        context_intro = f"You are analyzing performance logs for a SPECIFIC Spark Application: {app_id}."
    else:
        context_intro = "You are analyzing a fleet of Spark Applications. The logs provided may belong to different applications."

    system_prompt = f"""
    You are an expert Apache Spark Log Diagnostician. 
    {context_intro}

    Here are the most relevant log summaries retrieved from the system:
    
    {context_str}

    USER QUESTION: "{user_query}"

    INSTRUCTIONS:
    1. Identify root causes based ONLY on the provided log summaries.
    2. If analyzing multiple apps, clearly state which Application ID had which issue.
    3. If the logs mention 'Data Skew', 'GC Pressure', or 'Shuffle', explain what that means.
    4. Suggest concrete configuration changes where applicable.
    5. Keep the tone professional and concise.
    """

    # 3. GENERATION
    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(system_prompt)
        return response.text
    except Exception as e:
        return f"❌ Error communicating with Gemini API: {e}"

# ==========================================
# 4. MAIN EXECUTION - UPDATED
# ==========================================

if __name__ == "__main__":
    # A. Setup paths
    LOG_DIR = r"C:\Users\ddev\Documents\projects\spark-event-logs\test-logs"
    
    if not os.path.exists(LOG_DIR):
        print(f"❌ Error: Log directory '{LOG_DIR}' does not exist.")
        exit()

    # B. Ingest Logs
    ingest_logs(LOG_DIR)
    
    # C. Get the collection
    collection = get_chroma_collection()
    
    # D. Interactive Chat Loop
    print("\n" + "="*50)
    print("🤖 SPARK DIAGNOSTIC CHATBOT READY")
    print("="*50)
    
    print("Commands:")
    print("  - Type your question to search ALL logs.")
    print("  - Type 'set app <AppID>' to focus on one application.")
    print("  - Type 'clear app' to return to global search.")
    print("  - Type 'quit' to exit.\n")

    current_context_app = None

    while True:
        # Show current context in the prompt
        prompt_prefix = f"[{current_context_app}] You: " if current_context_app else "[ALL APPS] You: "
        user_input = input(prompt_prefix).strip()

        # Exit
        if user_input.lower() in ['quit', 'exit']:
            break
        
        # Command: Set Context
        if user_input.lower().startswith("set app "):
            parts = user_input.split(" ")
            if len(parts) > 2:
                current_context_app = parts[2].strip()
                print(f"🔒 Context locked to: {current_context_app}")
            continue

        # Command: Clear Context
        if user_input.lower() == "clear app":
            current_context_app = None
            print(f"🔓 Context cleared. Searching all logs.")
            continue
        
        # Standard Query
        answer = get_gemini_response(user_input, collection, app_id=current_context_app)
        print(f"\nGemini:\n{answer}\n")

📚 Vector DB already contains 789 documents. Skipping ingestion.
To force re-ingestion, delete the 'spark_rag_db' folder.

🤖 SPARK DIAGNOSTIC CHATBOT READY
Commands:
  - Type your question to search ALL logs.
  - Type 'set app <AppID>' to focus on one application.
  - Type 'clear app' to return to global search.
  - Type 'quit' to exit.



[ALL APPS] You:  what are the shortest running applications ? 


🔍 Searching across ALL application logs...

Gemini:
Based on the provided log summaries, it is not possible to determine the shortest running applications, as the logs only provide performance summaries for individual stages within applications, not the overall application runtime.

However, an analysis of the provided stage performance summaries reveals a common bottleneck across multiple applications:

**Root Cause:**
All identified applications are experiencing a **DESERIALIZATION_BOTTLENECK** within various stages. This indicates that a significant portion of the task runtime is spent converting serialized data (e.g., data received during shuffles or read from storage) back into Java objects.

**Application-Specific Issues:**

*   **app-20211011151946-6418:** Stage 4 exhibits high deserialization time (80.2% of runtime).
*   **app-20211011153421-6435:** Stage 0 exhibits high deserialization time (64.7% of runtime).
*   **app-20211012224801-11445:** Stage 1 exhibits high deserializa

[ALL APPS] You:  which apps ran the longest ? 


🔍 Searching across ALL application logs...

Gemini:
Based on the provided log summaries, the applications that ran the longest are:

*   **app-20211011231944-7649**: Ran for 10.32 seconds.
*   **app-20211011174919-6767**: Ran for 9.64 seconds.
*   **app-20211012230523-11486**: Ran for 9.64 seconds.

All applications completed successfully with no reported issues. Therefore, no root cause analysis or configuration changes are suggested.



[ALL APPS] You:  can you compare the longest and shortest running application ? 


🔍 Searching across ALL application logs...

Gemini:
Based on the provided log summaries:

The longest running application is **app-20211011231944-7649**, which completed Job 1 in 10.32 seconds.

The shortest running applications are **app-20211011151817-6416** and **app-20211011152324-6422**, both of which completed Job 1 in 9.32 seconds.

Comparing the longest and shortest runs, there is a difference of approximately 1 second (10.32s vs. 9.32s).

**Analysis:**
All applications reported a "JobSucceeded" status and no issues were identified in their respective `JobPerformanceSummary` logs. The observed variance in execution time is minimal (approximately 1 second difference between the longest and shortest runs) and does not indicate any performance degradation or specific root causes such as Data Skew, GC Pressure, or Shuffle issues based solely on these summaries.

**Recommendations:**
Given that all jobs succeeded and the performance variance is small, no immediate configuration chan

KeyboardInterrupt: Interrupted by user

In [5]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.
